<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ml801/marefa-mt-en-ar-parallel-10k-splitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!apt install git-lfs

from huggingface_hub import notebook_login
notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset Preprocess

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/arb-eng-parallel")

In [ ]:
one_mill_data = dataset['train'].shuffle(seed=42).select(range(1000000))
one_hundred_thousand_data = dataset['train'].shuffle(seed=42).select(range(100000))
ten_thousand_data = dataset['train'].shuffle(seed=42).select(range(10000))

In [ ]:
one_mill_data.push_to_hub("arb-eng-parallel-1mill")
one_hundred_thousand_data.push_to_hub("arb-eng-parallel-100k")
ten_thousand_data.push_to_hub("arb-eng-parallel-10k")

In [ ]:
from datasets import DatasetDict

def splitting_dataset(dataset):
  train_test = dataset.train_test_split(test_size=0.2)
  test_val = train_test['test'].train_test_split(test_size=0.5)

  train_set = train_test['train']
  val_set = test_val['train']
  test_set = test_val['test']

  combined_dataset = DatasetDict({
    'train': train_set,
    'validation': val_set,
    'test': test_set})

  return combined_dataset

In [ ]:
one_mill_data = splitting_dataset(one_mill_data)
one_hundred_thousand_data = splitting_dataset(one_hundred_thousand_data)
ten_thousand_data = splitting_dataset(ten_thousand_data)

In [ ]:
one_mill_data.push_to_hub("arb-eng-parallel-1mill-splitted")
one_hundred_thousand_data.push_to_hub("arb-eng-parallel-100k-splitted")
ten_thousand_data.push_to_hub("arb-eng-parallel-10k-splitted")

## Fine-tuning Marefa

In [3]:
!pip install transformers torch
!pip install transformers[torch]

In [4]:
!pip3 install transformers==4.3.0 sentencepiece==0.1.95 nltk==3.5 protobuf==3.15.3 torch==1.7.1

  Using cached transformers-4.3.0-py3-none-any.whl (1.8 MB)
  Using cached sentencepiece-0.1.95.tar.gz (508 kB)
  Preparing metadata (setup.py) ... done
  Using cached nltk-3.5.zip (1.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached protobuf-3.15.3-py2.py3-none-any.whl (173 kB)
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1)
ERROR: No matching distribution found for torch==1.7.1


In [3]:
!pip install sentencepiece

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("marefa-nlp/marefa-mt-en-ar")
model = AutoModelForSeq2SeqLM.from_pretrained("marefa-nlp/marefa-mt-en-ar")

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# dataset = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted")
dataset_cosine = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted-cosine-6")
dataset_euclidean = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted-euclidean-90")
dataset_translated_cosine = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted-translated-arabic-cosine-95")

In [17]:
def max_length_calculator(examples, column_name):
    return max(len(tokenizer.encode(ex)) for ex in examples[column_name])

max_length_train_english = max_length_calculator(dataset_euclidean['train'], 'english')
max_length_train_arabic = max_length_calculator(dataset_euclidean['train'], 'arabic')

max_length_validate_english = max_length_calculator(dataset_euclidean['validation'], 'english')
max_length_validate_arabic = max_length_calculator(dataset_euclidean['validation'], 'arabic')

highest_length = max(max_length_train_english, max_length_train_arabic, max_length_validate_english, max_length_validate_arabic)

In [18]:
from transformers import DataCollatorForSeq2Seq

def tokenize_function(example):
    source = example["english"]
    target = example["arabic"]

    # Tokenizing source and target without returning tensors and without padding
    tokenized_source = tokenizer(source, truncation=True, max_length=512, return_tensors="pt")
    tokenized_target = tokenizer(target, truncation=True, max_length=512, return_tensors="pt")

    return {
        "input_ids": tokenized_source["input_ids"][0],
        "attention_mask": tokenized_source["attention_mask"][0],
        "labels": tokenized_target["input_ids"][0]
    }

tokenized_datasets = dataset_euclidean.map(tokenize_function)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Map:   0%|          | 0/3141 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    output_dir="marefa-mt-en-ar-parallel-10k-splitted-euclidean",
    num_train_epochs=12,
    push_to_hub=True)

training_args = training_args.set_save(steps=10000)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [21]:
trainer.train()

Step,Training Loss
500,0.452900
1000,0.378000
1500,0.308900
2000,0.258400
2500,0.220600
3000,0.192100
3500,0.171600
4000,0.151600
4500,0.141000


TrainOutput(global_step=4716, training_loss=0.24736922670967806, metrics={'train_runtime': 8683.2644, 'train_samples_per_second': 4.341, 'train_steps_per_second': 0.543, 'total_flos': 802932561543168.0, 'train_loss': 0.24736922670967806, 'epoch': 12.0})

In [22]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1700908071.f5e25212822d.8181.3:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

'https://huggingface.co/HamdanXI/marefa-mt-en-ar-parallel-10k-splitted-euclidean/tree/main/'